# weibo 크롤링

## 크롤링 조건 설정

In [1]:
# 페이지 스크롤 다운 횟수 설정(숫자가 클수록 많은 페이지 크롤링)
num_scrolldown = 2
# 날짜 설정 : "설정 날짜" 포함, 이후 날짜 데이터 출력됨.
start_date = '2022-01-01'

In [2]:
from bs4 import BeautifulSoup as bs
import time
import random
from selenium import webdriver
import pandas as pd
from datetime import datetime
import getpass

## 페이지 로그인

In [3]:
# 사이트 로그인
driver = webdriver.Chrome('../chromedriver')
driver.get("https://weibo.com/login.php")
time.sleep(5)
driver.implicitly_wait(15)

# 창 최대화
driver.maximize_window()
time.sleep(1)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_17128/1712835742.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('../chromedriver')


In [4]:
# 로그인 정보
id = input("ID를 입력하세요 : ")
password = getpass.getpass("password(비밀번호)를 입력하세요 : ")

element_id = driver.find_element_by_xpath('//*[@id="loginname"]')
time.sleep(1)
element_password = driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[2]/div/input')

element_id.clear()
element_password.clear()

element_id.send_keys(id)
time.sleep( random.uniform(0.5, 1) )
element_password.send_keys(password)
time.sleep( random.uniform(0.5, 1) )

##로그인버튼 클릭
driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()
time.sleep(5)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_17128/3677500735.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_id = driver.find_element_by_xpath('//*[@id="loginname"]')
C:\Users\kbjung\AppData\Local\Temp/ipykernel_17128/3677500735.py:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_password = driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[2]/div/input')
C:\Users\kbjung\AppData\Local\Temp/ipykernel_17128/3677500735.py:18: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()


## 크롤링할 페이지 접속

In [5]:
# 페이지 접속
url = 'https://weibo.com/p/1001066011831795/home?from=page_100106&mod=TAB#place'
# webdriver 로드
driver.get(url)
time.sleep(5)
driver.implicitly_wait(5)

## 페이지 스크롤 다운

In [6]:
# 페이지 스크롤 다운
for i in range(num_scrolldown):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep( random.uniform(1,3))
    driver.implicitly_wait(5)

In [7]:
# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
print("타이틀 : ", soup.title.text)

타이틀 :  京畿道官博的微博_微博


## 데이터 크롤링

### 타이틀 수집

In [8]:
# 타이틀 수집
# 타이틀 첫번째 마침표까지만 가져오기
raw_contents = soup.find_all('div', class_='WB_text W_f14')
content_list = []
num = len(raw_contents)
for i in range(num):
    content = raw_contents[i].text.strip().split('。')[0]
    content_list.append(content)
    
print('원본 게시글 수 : ', len(raw_contents))
print('게시글 수 : ', len(content_list))

원본 게시글 수 :  45
게시글 수 :  45


### 조회수 수집

In [9]:
# 조회수 수집
raw_number_view = soup.find_all('i', class_='S_txt2')
number_view_list = []
for i in range(len(raw_number_view)):
    temp = raw_number_view[i].attrs['title']
    number_view = int(temp[9:-1])
    number_view_list.append(number_view)
    
print('조회수 리스트 원소수 : ', len(number_view_list))

조회수 리스트 원소수 :  45


### 좋아요, 공유수, 댓글 수 수집

In [10]:
# 정보 수집
raw_info = soup.find_all('span', 'line S_line1')

In [11]:
# 좋아요 수 수집
number_like_list = []
num_range = (len(raw_info) - 3) // 4 + 1
raw_info = soup.find_all('span', 'line S_line1')
for i in range(num_range):
    num = ( i + 1 ) * 4 - 1
    temp = raw_info[num]
    temp = temp.find_all('em')[-1].text
    number_like = int(temp)
    number_like_list.append(number_like)
    
print('좋아요 리스트 원소수 : ', len(number_like_list))

좋아요 리스트 원소수 :  45


In [12]:
# 공유 수 수집
number_share_list = []
num_range = (len(raw_info) - 3) // 4 + 1
raw_info = soup.find_all('span', 'line S_line1')
for i in range(num_range):
    num = ( i + 1 ) * 4 - 3
    temp = raw_info[num]
    temp = temp.find_all('em')[-1].text
    if '转发' in temp:
        number_share_list.append(0)
    else:
        number_like = int(temp)
        number_share_list.append(number_like)
        
print('공유수 리스트 원소수 : ', len(number_share_list))

공유수 리스트 원소수 :  45


In [13]:
# 댓글수 수집
number_comment_list = []
num_range_like = (len(raw_info) - 3) // 4 + 1
raw_info = soup.find_all('span', 'line S_line1')
for i in range(num_range_like):
    num = ( i + 1 ) * 4 - 2
    temp = raw_info[num]
    temp = temp.find_all('em')[-1].text
    if '评论' in temp:
        number_comment_list.append(0)
    else:
        number_like = int(temp)
        number_comment_list.append(number_like)
        
print('댓글수 리스트 원소수 : ', len(number_comment_list))

댓글수 리스트 원소수 :  45


### 게시글 날짜 수집

In [14]:
# 게시글 날짜 수집
date_list = []
raw_date = soup.find_all('div', 'WB_from S_txt2')
for i in range(len(raw_date)):
    temp = raw_date[i].find('a', 'S_txt2')
    date = temp.attrs['title'][:10]
    date_list.append(date)
    
print('날짜 리스트 원소수 : ', len(date_list))

날짜 리스트 원소수 :  45


### 팔로워 수 수집

In [15]:
# 팔로워 수 수집
number_follower_list = []
number_of_follower = soup.find_all('strong', 'W_f16')
num = int(number_of_follower[1].text)
number_follower_list.append(num)
for i in range(len(date_list)-1):
    number_follower_list.append(0)
    
print('팔로워 수 리스트 원소수 : ', len(number_follower_list))

팔로워 수 리스트 원소수 :  45


### 게시글 링크 수집

In [16]:
# 링크 수집
link_list = []
raw_links = soup.find_all('div', 'WB_from S_txt2')
for link in raw_links:
    temp = 'https://weibo.com' + link.find('a' ,'S_txt2').attrs['href']
    link_list.append(temp)
    
print("링크 리스트 원소수 : ", len(link_list))

링크 리스트 원소수 :  45


## 크롤링 데이터 확인

In [17]:
print('원본 게시글 수 : ', len(raw_contents))
print('게시글 수 : ', len(content_list))
print('조회수 리스트 원소수 : ', len(number_view_list))
print('좋아요 리스트 원소수 : ', len(number_like_list))
print('공유수 리스트 원소수 : ', len(number_share_list))
print('댓글수 리스트 원소수 : ', len(number_comment_list))
print('날짜 리스트 원소수 : ', len(date_list))
print('팔로워 수 리스트 원소수 : ', len(number_follower_list))
print("링크 리스트 원소수 : ", len(link_list))

원본 게시글 수 :  45
게시글 수 :  45
조회수 리스트 원소수 :  45
좋아요 리스트 원소수 :  45
공유수 리스트 원소수 :  45
댓글수 리스트 원소수 :  45
날짜 리스트 원소수 :  45
팔로워 수 리스트 원소수 :  45
링크 리스트 원소수 :  45


In [18]:
# chromedriver 종료
driver.quit()

## 데이터 셋 생성

In [19]:
# 데이터 프레임 생성
df = {'날짜':date_list, '제목':content_list, '조회수':number_view_list, '좋아요_수':number_like_list, '공유수':number_share_list, '댓글수':number_comment_list, '팔로워_수':number_follower_list, '링크':link_list}
data = pd.DataFrame(df)
data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수,링크
0,2022-04-22,让我们成为青少年主人公吧！ 一山Gogo乐园复古风的旱冰场！仅在京畿道可见的Gogo乐园，位...,2685,3,0,0,537949,https://weibo.com/6011831795/LpCigl2Hq?from=pa...
1,2022-04-20,充满回忆，具有复古感性的杨平青春博物馆你可以感受到过去的复古感在杨平青春博物馆享受复古风吧,3382,4,0,0,0,https://weibo.com/6011831795/LpkAuxpHJ?from=pa...
2,2022-04-18,适合春天去的杨州张旭镇美术馆位于扬州的市立张旭镇美术馆，雕塑公园内不仅有展览公园也是非常适合...,8427,3,1,0,0,https://weibo.com/6011831795/Lp1Xtv4gk?from=pa...
3,2022-04-15,京畿道代表庆典， 军浦杜鹃花节每年四月，在京畿道军埔市杜鹃花乐园会有很多游客,7684,2,0,0,0,https://weibo.com/6011831795/LoztQBBCV?from=pa...
4,2022-04-13,在亚洲三大设计大奖上获奖的工厂型特色咖啡厅京畿道 坡州市的代表人气场所的大型一体工厂型咖啡厅...,7164,7,1,0,0,https://weibo.com/6011831795/LofTX2JNh?from=pa...
5,2022-04-11,韩国巧克力研究所博物馆介绍和体验消息随着巧克力产业的发展，巧克力制作师成为专业职业,9800,5,3,0,0,https://weibo.com/6011831795/LnXD6737G?from=pa...
6,2022-04-08,杨平两水里韩国观光100选中经常被选的京畿道杨平两水里,8865,4,0,0,0,https://weibo.com/6011831795/Lnu0iiELi?from=pa...
7,2022-04-06,韩国有名的铁板烧烤店，高阳市大型铁板烧烤喜欢探访美食店的明星们喜欢的美食店，炭烧铁板烧烤位于...,10302,8,2,4,0,https://weibo.com/6011831795/LncCEkEJA?from=pa...
8,2022-04-04,利川市特色咖啡厅西蒙斯露台京畿道利川市有着世界级床品牌西蒙斯工厂,11672,2,0,0,0,https://weibo.com/6011831795/LmUDQ3zI8?from=pa...
9,2022-04-01,"坡州市游戏云,从废弃的设施迅速崛起为新的乐园京畿道坡州市中央生态公园作为拥有湖泊公园和草坪广...",11935,4,0,0,0,https://weibo.com/6011831795/Lmr9ashHn?from=pa...


## 데이터 셋. 날짜로 슬라이싱

In [20]:
# 데이터 셋 날짜로 슬라이싱
slicing_data = data[data['날짜'] >= start_date]
slicing_data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수,링크
0,2022-04-22,让我们成为青少年主人公吧！ 一山Gogo乐园复古风的旱冰场！仅在京畿道可见的Gogo乐园，位...,2685,3,0,0,537949,https://weibo.com/6011831795/LpCigl2Hq?from=pa...
1,2022-04-20,充满回忆，具有复古感性的杨平青春博物馆你可以感受到过去的复古感在杨平青春博物馆享受复古风吧,3382,4,0,0,0,https://weibo.com/6011831795/LpkAuxpHJ?from=pa...
2,2022-04-18,适合春天去的杨州张旭镇美术馆位于扬州的市立张旭镇美术馆，雕塑公园内不仅有展览公园也是非常适合...,8427,3,1,0,0,https://weibo.com/6011831795/Lp1Xtv4gk?from=pa...
3,2022-04-15,京畿道代表庆典， 军浦杜鹃花节每年四月，在京畿道军埔市杜鹃花乐园会有很多游客,7684,2,0,0,0,https://weibo.com/6011831795/LoztQBBCV?from=pa...
4,2022-04-13,在亚洲三大设计大奖上获奖的工厂型特色咖啡厅京畿道 坡州市的代表人气场所的大型一体工厂型咖啡厅...,7164,7,1,0,0,https://weibo.com/6011831795/LofTX2JNh?from=pa...
5,2022-04-11,韩国巧克力研究所博物馆介绍和体验消息随着巧克力产业的发展，巧克力制作师成为专业职业,9800,5,3,0,0,https://weibo.com/6011831795/LnXD6737G?from=pa...
6,2022-04-08,杨平两水里韩国观光100选中经常被选的京畿道杨平两水里,8865,4,0,0,0,https://weibo.com/6011831795/Lnu0iiELi?from=pa...
7,2022-04-06,韩国有名的铁板烧烤店，高阳市大型铁板烧烤喜欢探访美食店的明星们喜欢的美食店，炭烧铁板烧烤位于...,10302,8,2,4,0,https://weibo.com/6011831795/LncCEkEJA?from=pa...
8,2022-04-04,利川市特色咖啡厅西蒙斯露台京畿道利川市有着世界级床品牌西蒙斯工厂,11672,2,0,0,0,https://weibo.com/6011831795/LmUDQ3zI8?from=pa...
9,2022-04-01,"坡州市游戏云,从废弃的设施迅速崛起为新的乐园京畿道坡州市中央生态公园作为拥有湖泊公园和草坪广...",11935,4,0,0,0,https://weibo.com/6011831795/Lmr9ashHn?from=pa...


# 데이터 셋. 인덱스 내림차순 정렬

In [21]:
# 인덱스 내림차순 정렬
final_data = slicing_data.sort_index(ascending=False)
final_data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수,링크
43,2022-01-01,🎊 新年快乐! 🎊祝你2022年与京畿道一起度过幸福的一年,12110,11,0,4,0,https://weibo.com/6011831795/L8IJk9tQd?from=pa...
42,2022-01-05,京畿道支援的异色创业趋势！充分利用共享厨房的 “G-创食学院”的运营共享厨房是指多个营业者共...,14056,3,0,0,0,https://weibo.com/6011831795/L9mdF9t5G?from=pa...
41,2022-01-06,美味的冬季京畿道传统市场一到冬天就会想起鲫鱼饼、烤红薯、鱼糕、甜饼等很多零食吧？请来访问具有...,15741,5,0,0,0,https://weibo.com/6011831795/L9wcfEBPu?from=pa...
40,2022-01-07,🍅京畿道光州市代表农作物番茄的重新诞生！酸酸甜甜的番茄辣椒酱同大韩民国代表酱料辣椒酱一样受到...,16359,4,0,1,0,https://weibo.com/6011831795/L9FbKuzBl?from=pa...
39,2022-01-13,京畿道为外国人居民定居支援的事业募集📣京畿道，支援外国人居民与高丽人同胞支援事业🔥在大韩民国...,16139,5,1,1,0,https://weibo.com/6011831795/LaAyPh9ZV?from=pa...
38,2022-01-15,MZ世代的元宇宙元宇宙图书馆，京畿道城南教育图书馆在京畿道有了元宇宙元宇宙图书馆,17044,3,0,0,0,https://weibo.com/6011831795/LaMlxesju?from=pa...
37,2022-01-17,京畿道平泽市，2021平泽世界文化周成功落下帷幕🎉连接世界的文化散策城市 🏃在京畿道平泽市举...,18293,5,1,1,0,https://weibo.com/6011831795/Lbck79XJz?from=pa...
36,2022-01-20,🎹 风乐响起来!! 🎺🎺在京畿道的博物馆都去过哪里？@@(学习历史最好的地方👍👍)👉京畿道博...,19750,3,1,0,0,https://weibo.com/6011831795/LbCqQmNzF?from=pa...
35,2022-01-24,主题公园，都去过哪里呢介绍一下位于京畿道的国内最大的汽车主题公园,19832,2,0,0,0,https://weibo.com/6011831795/LcdTRzXmm?from=pa...
34,2022-01-26,拥有很多世界文化遗产的美丽的水原华城2021年在华城举办了‘媒体艺术秀’,19109,3,1,1,0,https://weibo.com/6011831795/Lcyygmjkp?from=pa...


## 엑셀 파일로 출력

In [22]:
# 파일로 출력
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'weibo_' + file_name + '.xlsx'
path = 'crawled_data/weibo/'
final_data.to_excel(path+file_name, index=False, encoding='utf-8')

print(f"{file_name} 파일 생성 완료")

weibo_20220425-095237.xlsx 파일 생성 완료


# 코드 마지막